# **Customer Purchase Analysis**

---



*   Customer Analysis:

  1.   Repeat purchase rate
  2.   Churn rate
  3.   Retention rate
  4.   Customer Lifetime Value (CLTV)
  5.   Revenue impact of customer retentio



*   Sales Performance Analysis:

  1.   Monthly and yearly sales trends
  2.   Products with the highest frequency
  3.   Revenue analysis based on country
  4.   Average revenue per transaction
  5.   Popular days of the week for purchases




In [81]:
!wget https://archive.ics.uci.edu/static/public/352/online+retail.zip

--2024-08-17 18:09:16--  https://archive.ics.uci.edu/static/public/352/online+retail.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘online+retail.zip.1’

online+retail.zip.1     [         <=>        ]  22.62M  4.56MB/s    in 5.2s    

2024-08-17 18:09:22 (4.34 MB/s) - ‘online+retail.zip.1’ saved [23715478]



In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
!unzip online+retail.zip

Archive:  online+retail.zip
replace Online Retail.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

In [ ]:
# Install the necessary library

import pandas as pd

# Load the Excel file
data = pd.read_excel('Online Retail.xlsx')

# Save DataFrame to CSV
data.to_csv('OnlineRetail.csv', index=False)

# Verify by listing files
!ls


In [ ]:
data = pd.read_csv('OnlineRetail.csv')
data

In [ ]:
data.shape

The current dataset has around 541909 rows and 8 different columns

In [ ]:
#Exploring around different columns
data.columns

In [ ]:
data.info()

From the information type we can observe some columns are not in correct dtype and format

*   Invoice Number which is integer but is shown as "Object"
*   InvoiceDate which is datetime is shown as "Object"


In [ ]:
#Primary Goal ---> converting to correct format.
#Targeted Columns ---> InvoiceDate and InvoiceNumber

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceNo'] = data['InvoiceNo'].astype(int)


We can see that invoiceNo which seems to be int is not int and is also not getting convert to int.
checking if it actually is a integer based column or not

In [ ]:
non_numeric_mask = ~data['InvoiceNo'].astype(str).str.isnumeric()
non_numeric_values = data[non_numeric_mask]
non_numeric_values

As we can see that there is string present in the avobe dataset for the column invoice number. SO, we cannot convert it

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data_copy = data.copy()
data_copy

In [ ]:
data_copy.head(3)

This report covers a detailed analysis about custoer purchase and a detailed analysis about how the factors are dependent on each other.

In [ ]:
#Task: Calculate the total revenue generated by each country.
data_copy['TotalRevenue'] = data_copy['Quantity'] * data_copy['UnitPrice']
revenue_analysis = data_copy.groupby('Country')['TotalRevenue'].sum().sort_values(ascending = False).reset_index()
revenue_analysis

In [ ]:
top_countries = revenue_analysis.head(5)
top_countries

In [ ]:
#creating another dummy dataset for storing the revenue of the countries that are not in top 5

others = pd.DataFrame({'Country':'Others','TotalRevenue':[revenue_analysis['TotalRevenue'].iloc[5:].sum()]})
others

In [ ]:
#To create a versatie pie chat to show the shares captured by top countries and others

combined_revenue_analysis = pd.concat([top_countries, others], ignore_index=True)
combined_revenue_analysis

In [ ]:
#Versatile Pie Chat
combined_revenue_analysis.set_index('Country', inplace=True)
combined_revenue_analysis['TotalRevenue'].plot(kind='pie', autopct='%1.1f%%', figsize=(10, 8))


In [ ]:
data_copy.head()

In [ ]:
#Compute the total amount spent by each customer.
consumer_spending_analysis = data_copy.groupby('CustomerID')['TotalRevenue'].sum().reset_index()
consumer_spending_analysis

In [ ]:
#Identify Top 10 customers based on TotalRevenue:
top_customers = consumer_spending_analysis.sort_values(by = 'TotalRevenue', ascending = False).head(10)
top_customers

In [ ]:
#Identify The transaction made by each customers:
transaction_count = data_copy.groupby('CustomerID')['InvoiceNo'].nunique().reset_index(name='TransactionCount')
transaction_count

In [ ]:
#Merging the above data to top10
top10_customers = top_customers.merge(transaction_count, on='CustomerID')
top10_customers

In [ ]:
#Calcuationg the average transaction per customer
top10_customers['AverageSpendingPerTransaction'] = top10_customers['TotalRevenue'] / top10_customers['TransactionCount']
top10_customers['AverageSpendingPerTransaction']

In [ ]:
data_copy.head(4)

In [ ]:
# Analyze the sales performance over time.
#Specifically, calculate the total revenue generated per month, and identify any trends or patterns.
#Additionally, determine which month had the highest sales and which had the lowest.
data_copy['month'] = data_copy['InvoiceDate'].dt.month
sales_performance_analysis = data_copy.groupby('month')['TotalRevenue'].sum().reset_index()
sales_performance_analysis

In [ ]:
#Checking for the month that has observed the maximum revenue and the month with minium revenue
max_sales_performance_analysis = sales_performance_analysis['TotalRevenue'].max()
min_sales_performance_analysis = sales_performance_analysis['TotalRevenue'].min()

#Extracting the month associated with the max values
month_max_sales_performance_analysis = sales_performance_analysis.loc[sales_performance_analysis['TotalRevenue'].idxmax()]
month_min_sales_performance_analysis = sales_performance_analysis.loc[sales_performance_analysis['TotalRevenue'].idxmin()]
#observing the values
print(f"Highest Revenue: Month {month_max_sales_performance_analysis['month']} with ${month_max_sales_performance_analysis['TotalRevenue']:.2f}")
print(f"Highest Revenue: Month {month_min_sales_performance_analysis['month']} with ${month_min_sales_performance_analysis['TotalRevenue']:.2f}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(sales_performance_analysis['month'], sales_performance_analysis['TotalRevenue'], marker='o', linestyle='-', color='b', linewidth=2)

# Highlighting the month with the highest revenue
plt.plot(month_max_sales_performance_analysis['month'], month_max_sales_performance_analysis['TotalRevenue'], 'go', markersize=10, label='Max Revenue')

# Highlighting the month with the lowest revenue
plt.plot(month_min_sales_performance_analysis['month'], month_min_sales_performance_analysis['TotalRevenue'], 'ro', markersize=10, label='Min Revenue')

# Adding title and labels
plt.title('Monthly Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Total Revenue')
plt.xticks(sales_performance_analysis['month'])  # Ensure all months are shown on the x-axis

# Adding a legend
plt.legend()

# Display the plot
plt.grid(True)
plt.show()

In [ ]:
# Analyze the sales performance over time.
#Specifically, calculate the total revenue generated per YEAR, and identify any trends or patterns.
#Additionally, determine which month had the highest sales and which had the lowest.
data_copy['year'] = data_copy['InvoiceDate'].dt.year
year_performance_analysis = data_copy.groupby('year')['TotalRevenue'].sum().reset_index()
year_performance_analysis

In [ ]:
data_copy['year'].value_counts()

In [ ]:
max_year_performance_analysis = year_performance_analysis.loc[year_performance_analysis['TotalRevenue'].idxmax()]
min_year_performance_analysis = year_performance_analysis.loc[year_performance_analysis['TotalRevenue'].idxmin()]

print(f"Highest Revenue: Year {max_year_performance_analysis ['year']} with ${max_year_performance_analysis['TotalRevenue']:.2f}")
print(f"Highest Revenue: Year {min_year_performance_analysis ['year']} with ${min_year_performance_analysis['TotalRevenue']:.2f}")

In [ ]:
# Plotting the revenue trend by year
plt.figure(figsize=(5, 5))

# Plotting the total revenue per year
plt.plot(year_performance_analysis['year'], year_performance_analysis['TotalRevenue'], marker='o', linestyle='-', color='b', label='Revenue')

# Highlighting the year with maximum revenue
plt.plot(max_year_performance_analysis['year'], max_year_performance_analysis['TotalRevenue'], 'go', markersize=10, label='Max Revenue')

# Highlighting the year with minimum revenue
plt.plot(min_year_performance_analysis['year'], min_year_performance_analysis['TotalRevenue'], 'ro', markersize=10, label='Min Revenue')

# Adding title and labels
plt.title('Yearly Revenue Trend')
plt.xlabel('Year')
plt.ylabel('Total Revenue')
plt.xticks(year_performance_analysis['year'])  # Ensure all years are shown on the x-axis

# Adding a legend
plt.legend()

# Display the plot
plt.grid(True)
plt.show()

In [ ]:
data_copy.head(4)


In [ ]:
#Identify the most frequently purchased products.

#Task:
#Find the top 10 products with the highest purchase frequency.
#Display the product descriptions and the number of times each product was purchased.

product_with_highest_frequency = data_copy.groupby('Description')['TotalRevenue'].size().reset_index(name='Frequency')
product_with_highest_frequency.sort_values(by='Frequency', ascending=False).head(10)

In [ ]:
#Analyze the sales performance by country.

#Task:
#Calculate the total revenue generated by each country.
#Identify the top 5 countries with the highest revenue.
#Plot a bar chart to visualize the revenue distribution among these top 5 countries.
import seaborn as sns
revenue_analysis_based_on_country = data_copy.groupby('Country')['TotalRevenue'].sum().reset_index()
top_5_countries = revenue_analysis_based_on_country.sort_values(by = 'TotalRevenue', ascending = False).head(5)
sns.barplot(x='TotalRevenue', y='Country', data=top_5_countries)
plt.show()


In [ ]:
#Caculate the average revenue per transaction
revenue_per_transcation = data_copy.groupby('InvoiceNo')['TotalRevenue'].mean().reset_index()
revenue_per_transcation

In [ ]:
#Determine the average revenue per transaction.

#Task:
#Calculate the average revenue generated per transaction (InvoiceNo).
#Display the average revenue and the total number of transactions.

revenue_per_transactions = data_copy.groupby('InvoiceNo')['TotalRevenue'].sum().reset_index()
average_revenue_per_transaction = revenue_per_transactions['TotalRevenue'].mean()
total_transactions = revenue_per_transactions['InvoiceNo'].nunique()
total_transactions

In [ ]:
data_copy.head(3)

In [ ]:
data_copy['week'] = data_copy['InvoiceDate'].dt.day_name()
data_copy['week']

In [ ]:
data_copy.head(3)

In [ ]:
#Identify the most popular day of the week for purchases.

#Task:
#Add a new column to the dataset indicating the day of the week for each transaction (e.g., Monday, Tuesday).
#Determine which day of the week has the highest number of transactions.
#Display the day and the number of transactions.

data_copy['week'] = data_copy['InvoiceDate'].dt.day_name()
highest_transaction_day = data_copy.groupby('week')['InvoiceNo'].size().reset_index(name = 'Count')
topv = highest_transaction_day.loc[highest_transaction_day['Count'].idxmax()]  # Find the row with the highest count
print(f"The most popular day for purchases is {topv['week']} with {topv['Count']} transactions.")


In [ ]:
data_copy.head(1)

In [ ]:
#Identify the average revenue per transaction and compare it across different countries.

#Task:

#Calculate the average revenue per transaction for each country.
#Display the countries with the highest and lowest average revenue per transaction.
#Plot the average revenue per transaction for all countries.

country_revenue = data_copy.groupby('Country')['TotalRevenue'].sum()
country_transaction = data_copy.groupby('Country')['InvoiceNo'].nunique()
average_transaction_per_country = (country_revenue / country_transaction).reset_index(name = 'Average_transaction_per_country')
max_avg_revenue_country = average_transaction_per_country.loc[average_transaction_per_country['Average_transaction_per_country'].idxmax()]
min_avg_revenue_country = average_transaction_per_country.loc[average_transaction_per_country['Average_transaction_per_country'].idxmin()]

print(f"Country with highest average revenue per transaction: {max_avg_revenue_country['Country']} (${max_avg_revenue_country['Average_transaction_per_country']:.2f})")
print(f"Country with lowest average revenue per transaction: {min_avg_revenue_country['Country']} (${min_avg_revenue_country['Average_transaction_per_country']:.2f})")
plt.figure(figsize=(12,6))
plt.bar(average_transaction_per_country['Country'], average_transaction_per_country['Average_transaction_per_country'], color='skyblue')
plt.xticks(rotation=90)
plt.xlabel('Country')
plt.ylabel('Average Revenue per Transaction')
plt.title('Average Revenue per Transaction by Country')
plt.show()


In [ ]:
data_copy.head(3)

In [ ]:
#Objective: Calculate the repeat purchase rate and customer churn rate.
#Breaking down the part to two major part

#Repeat Purchase Rate

customer_repeat = data_copy.groupby('CustomerID')['InvoiceNo'].nunique()

repeat_frequency = customer_repeat[customer_repeat>1].count()

total_customer_frquency = customer_repeat.count()

repeat_purchase_rate = (repeat_frequency/total_customer_frquency)*100

#Customer Churn Rate

churn_customers = customer_repeat[customer_repeat == 1].count()
churn_rate = (churn_customers/total_customer_frquency)*100

In [ ]:
#Identify trends in customer retention over time.

# Identify the First Purchase Month for Each Customer
first_purchase_month = data_copy.groupby('CustomerID')['month'].min().reset_index()
first_purchase_month.columns = ['CustomerID', 'FirstPurchaseMonth']

# Merge this with the original data
data_copy = pd.merge(data_copy, first_purchase_month, on='CustomerID')
data_copy

In [ ]:
# Tracking Subsequent Purchases
data_copy['RepeatPurchase'] = (data_copy['month'] > data_copy['FirstPurchaseMonth']).astype(int)
data_copy['RepeatPurchase']


In [ ]:
#retention rate
retention_rate = data_copy.groupby('FirstPurchaseMonth')['RepeatPurchase'].mean().reset_index()
retention_rate.columns = ['CohortMonth', 'RetentionRate']

retention_rate.columns
retention_rate

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(retention_rate['CohortMonth'], retention_rate['RetentionRate'], marker='o')
plt.title('Customer Retention Rate Over Time')
plt.xlabel('Cohort Month')
plt.ylabel('Retention Rate')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
monthly_customers = data_copy.groupby('month')['CustomerID'].nunique().reset_index(name='UniqueCustomers')
data_copy['PrevYearMonth'] = data_copy['month'].shift(1)
data_copy.head(5)

In [ ]:
#Calculating Customer Lifetime Value (CLTV)

#Calculating average revenue per transactions
total_revenue_per_customer = data_copy['TotalRevenue'].sum()
total_tranaction_per_customer = data_copy['InvoiceNo'].nunique()
average_revenue_per_transactions = total_revenue_per_customer/total_tranaction_per_customer

#Calculating Average Number of Transactions per customer
customer_transactions = data_copy.groupby('CustomerID')['InvoiceNo'].nunique()
avg_transactions_per_customer = customer_transactions.mean()

#Calculate Customer Lifetime Value (CLTV)
cltv = average_revenue_per_transactions * avg_transactions_per_customer

print(f"Customer Lifetime Value (CLTV): ${cltv:.2f}")


In [ ]:
#Revenue Impact Analysis

# 1. Calculate Total Revenue
total_revenue = data_copy['TotalRevenue'].sum()

# 2. Calculate Retention Rate
# Define retention calculation based on your method
customer_ret = data_copy.groupby('CustomerID')['InvoiceNo'].nunique()
retention_rate = (customer_ret > 1).mean() * 100

# 3. Segment Customers Based on Retention Rate
# Define segments - adjust as needed based on your data
high_retention = customer_ret[customer_ret > customer_ret.median()]
low_retention = customer_ret[customer_ret <= customer_ret.median()]

# 4. Calculate Total Revenue for Each Segment
revenue_high_retention = data_copy[data_copy['CustomerID'].isin(high_retention.index)]['TotalRevenue'].sum()
revenue_low_retention = data_copy[data_copy['CustomerID'].isin(low_retention.index)]['TotalRevenue'].sum()

# Display Results
print(f"Total Revenue: ${total_revenue:.2f}")
print(f"Retention Rate: {retention_rate:.2f}%")
print(f"Revenue for High Retention Customers: ${revenue_high_retention:.2f}")
print(f"Revenue for Low Retention Customers: ${revenue_low_retention:.2f}")

In [ ]:
#Seasonality Analysis

# Aggregate sales by month and year
seasonal_sales = data_copy.groupby(['year', 'month'])['TotalRevenue'].sum().reset_index()

# Plot seasonal trends
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(x='month', y='TotalRevenue', hue='year', data=seasonal_sales)
plt.title('Seasonal Sales Trends')
plt.xlabel('month')
plt.ylabel('Total Revenue')
plt.grid(True)
plt.show()

In [ ]:
#Customer Purchase Frequency Analysis

purchase_frequency = data_copy.groupby('CustomerID')['InvoiceNo'].size().reset_index(name='PurchaseFrequency')
# Plot purchase frequency distribution
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(purchase_frequency['PurchaseFrequency'], bins=20, kde=True)
plt.title('Distribution of Purchase Frequency')
plt.xlabel('Number of Purchases')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Calculate AOV
aov = data_copy.groupby('InvoiceNo')['TotalRevenue'].sum().reset_index()
aov_mean = aov['TotalRevenue'].mean()

# Aggregate AOV by month
monthly_aov = data_copy.groupby(['year', 'month'])['TotalRevenue'].sum() / data_copy.groupby(['year', 'month'])['InvoiceNo'].nunique()
monthly_aov = monthly_aov.reset_index(name='AOV')

# Plot AOV trends
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(x='month', y='AOV', hue='year', data=monthly_aov)
plt.title('Average Order Value (AOV) Trends')
plt.xlabel('month')
plt.ylabel('Average Order Value')
plt.grid(True)
plt.show()

In [ ]:
# Sort by customer and date
data_copy = data_copy.sort_values(by=['CustomerID', 'InvoiceDate'])

# Calculate time between purchases
data_copy['TimeBetweenPurchases'] = data_copy.groupby('CustomerID')['InvoiceDate'].diff().dt.days

# Analyze the distribution of time between purchases
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(data_copy['TimeBetweenPurchases'].dropna(), bins=30, kde=True)
plt.title('Distribution of Time Between Purchases')
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.show()

# **Detailed Review of Insights**


# Sales Performance Analysis


---


Monthly and Yearly Sales Trends:

Monthly Trends: You analyzed the sales trends by month, identifying peak and low sales months. This insight helps understand seasonal variations and is crucial for strategic planning, inventory management, and marketing efforts.
Yearly Trends: By examining yearly sales data, you identified overall growth or decline trends, which can inform long-term strategic decisions and investment priorities.

Products with Highest Frequency:

Top Products: Identifying products with high purchase frequencies helps in focusing marketing efforts and ensuring stock availability. This insight is key for optimizing product listings and promotional strategies.
Low Frequency Products: Analyzing products with lower purchase frequencies allows you to assess their performance and decide if they need better marketing or should be discontinued.

Revenue Analysis Based on Country:

Top Revenue Generating Countries: This insight highlights the most lucrative markets, guiding where to focus sales and marketing efforts.
Low Revenue Countries: Understanding low-performing markets helps identify potential issues and areas for improvement.

Average Revenue per Transaction:

High Value Transactions: Recognizing regions or segments with high average revenue per transaction allows for targeted premium strategies.
Low Value Transactions: Identifying factors behind lower average revenue helps in implementing strategies to boost transaction values.

Popular Days of the Week for Purchases:

Peak Days: Knowing the days with the highest transaction volumes helps in optimizing staffing, promotions, and operational strategies.
Low Traffic Days: Understanding days with lower activity helps in crafting strategies to increase engagement and sales on those days.\


# Customer Analysis


---



Repeat Purchase Rate:

High Repeat Rate: Indicates strong customer loyalty and effectiveness of retention strategies. It’s essential to analyze and replicate successful practices to enhance overall customer retention.
Low Repeat Rate: Signals potential issues in customer satisfaction or engagement. It’s important to investigate and implement strategies to improve repeat purchases.

Churn Rate:

High Churn Rate: Identifying high churn rates points to problems with customer retention. Strategies to address customer dissatisfaction and improve retention are crucial.
Low Churn Rate: Reflects effective retention efforts. Maintaining and building upon successful practices can further enhance customer loyalty.

Retention Rate:

High Retention Rate: High retention rates indicate successful retention strategies. Emphasize and expand on successful tactics to maintain and increase retention.
Low Retention Rate: Highlights the need for improved retention strategies. Explore reasons for low retention and develop targeted actions to address them.

Customer Lifetime Value (CLTV):

High CLTV: Represents valuable customers contributing significantly to revenue over their lifetime. Focus on nurturing these relationships to maximize their value.
Low CLTV: Indicates the need to enhance strategies to increase the lifetime value of customers. Implementing upselling or cross-selling strategies may help improve CLTV.

Revenue Impact of Customer Retention:

Positive Impact: Higher retention rates positively influence revenue growth. Continue investing in retention strategies to leverage this impact further.
Neutral/Negative Impact: Analyze the effectiveness of current retention strategies and adjust them to ensure they contribute positively to revenue growth.